In [47]:
name_mapping = {
    'control': 'Control (Retrain)',
    'certified_deep_unlearning': 'Certified Deep Unlearning',
    'amnesiac': 'Amnesiac',
    'bad_teacher': 'Bad Teacher',
    'ssd': "SSD"
}

In [48]:
from pathlib import Path
from argparse import ArgumentParser
import pandas as pd
import numpy as np

experiment_dir = '/fendlnm1/mirror-mirror/epsilon_experiment/'
exp_dir = Path(experiment_dir)
distinguishers = [fp for fp in exp_dir.iterdir() if str(fp).endswith('csv')]
# print(distinguishers)
adv_data = []
all_means = []
for distinguisher_fp in distinguishers:
    if 'kld' in str(distinguisher_fp):
        distinguisher_data = pd.read_csv(distinguisher_fp)
        unlearning_approaches = distinguisher_data['model'].unique()
        assert 'control' in unlearning_approaches
        distinguisher_name = '_'.join(distinguisher_fp.name.split('_')[:-1])
        control_data = distinguisher_data[distinguisher_data['model'] == 'control']
        for ul_approach in unlearning_approaches:
            if ul_approach == 'control':
                ul_data = distinguisher_data[distinguisher_data['model'] == ul_approach]
                for exp in range(1,6):
                    sigma = 10**(-exp)
                    mean_scores = {'score': np.mean(ul_data['score']), 'std': np.std(ul_data['score']), 'method': name_mapping[ul_approach], 'distinguiser':distinguisher_fp.parts[-1][:-4], 'sigma':sigma}
                # mean_scores = {'mean': np.mean(ul_data['score']), 'std': np.std(ul_data['score']), 'method': ul_approach, 'distinguiser':distinguisher_fp.parts[-1][:-4]}
                    all_means.append(mean_scores)
                continue
            else:
                ul_data = distinguisher_data[distinguisher_data['model'] == ul_approach]
    
                greater_than = (ul_data['score'].values > control_data['score'].values).sum() / len(ul_data['score'].values)
                less_than = (ul_data['score'].values < control_data['score'].values).sum() / len(ul_data['score'].values)
                equal_to = (ul_data['score'].values == control_data['score'].values).sum() / len(ul_data['score'].values)
                win_percentage =  max(greater_than, less_than) + (equal_to / 2)
                advantage = (win_percentage - 0.5) * 2
                # win_percentage = max(greater_than, less_than)
                name_string = name_mapping[ul_approach.split('.')[0]]
                sigma = 10**(-(int(ul_approach.split('.')[1])))
                row_data = {
                    'distinguisher': distinguisher_name,
                    'advantage': advantage,
                    'unlearning': name_string,
                    'sigma': str(sigma),
                    'win_percentage': win_percentage,
                }
                # print(greater_than, less_than)
                adv_data.append(row_data)
                method = ul_approach.split('.')[0]
                exp = int(ul_approach.split('.')[1])
                sigma = 10**(-exp)
                mean_scores = {'score': np.mean(ul_data['score']), 'std': np.std(ul_data['score']), 'method': name_mapping[method], 'distinguiser':distinguisher_fp.parts[-1][:-4], 'sigma':sigma}
                all_means.append(mean_scores)
adv_df = pd.DataFrame.from_dict(adv_data)
mean_df = pd.DataFrame(all_means)
# mean_df.to_csv('graph_data.csv')
# print(distinguisher_data)


In [49]:
import plotly.express as px
kld = mean_df[mean_df['distinguiser'] == 'kld_results']
fig = px.line(
    kld,
    x='sigma',
    y='score',
    color='method',
    error_y='std',
    # title=f"Comparison of Certified Removal Sigma against {distinguisher.replace('_', ' ').upper()} Distinguisher Scores",
    width=900,
    height=450,
    markers=True,
    log_x=True,
    log_y=True,
    labels={
        'method': 'Unlearning Method',
        'score': "KLDScore",
        'sigma': r"$\text{Certified Deep Unlearning} \ \sigma \ \text{Parameter}$|",
    },
    category_orders={'method': ['Control (Retrain)', 'Certified Deep Unlearning']}
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    # paper_bgcolor="LightSteelBlue",
)

# fig.update_yaxes(minallowed=0)
fig.write_image('images/kld_cdu_sigma.png')
fig.write_image('images/kld_cdu_sigma.pdf')
fig.show()

In [50]:
fig_adv = px.bar(adv_df, x="sigma", y="win_percentage", barmode="stack", color="sigma",
    width=900,
    height=450,
    labels = {
        'win_percentage': 'Success Rate',    
        'model': 'Unlearning Method',
        'score': "MIAScore",
        'sigma': r"$\text{CDU} \ \sigma \ \text{Param}$"
    },
    )

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig_adv.update_yaxes(minallowed=0.5)
fig_adv.show()
fig_adv.write_image('images/sigma_adv.png')
fig_adv.write_image('images/sigma_adv.pdf')

In [51]:
from pathlib import Path
from argparse import ArgumentParser
import pandas as pd
import numpy as np

experiment_dir = '/fendlnm1/mirror-mirror/certified_forgetset_experiment/'
exp_dir = Path(experiment_dir)
distinguishers = [fp for fp in exp_dir.iterdir() if str(fp).endswith('csv')]
# print(distinguishers)
graph_data = []
all_means = []
adv_data = []
for distinguisher_fp in distinguishers:
    distinguisher_data = pd.read_csv(distinguisher_fp)
    # unlearning_approaches = distinguisher_data['model'].unique()
    # for ul_approach in unlearning_approaches:
    for forget_set_size in distinguisher_data['forget_set'].unique():
        control_data = distinguisher_data[np.array(distinguisher_data['model'] == 'control') & np.array(distinguisher_data['forget_set'] == forget_set_size)]
        for ul_approach in distinguisher_data['model'].unique():
            ul_string = name_mapping[ul_approach]
            condition = np.array(distinguisher_data['forget_set'] == forget_set_size) & np.array(distinguisher_data['model'] == ul_approach)
            ul_data = distinguisher_data[condition]
            graph_data.append({
                'score': ul_data['score'].mean(),
                'model': ul_string,
                'forget_set_size': forget_set_size.split('_')[-1],
                'std': ul_data['score'].std(),
            })
            if ul_approach != 'control':
                greater_than = (ul_data['score'].values > control_data['score'].values).sum() / len(ul_data['score'].values)
                less_than = (ul_data['score'].values < control_data['score'].values).sum() / len(ul_data['score'].values)
                equal_to = (ul_data['score'].values == control_data['score'].values).sum() / len(ul_data['score'].values)

                win_percentage =  max(greater_than, less_than) + (equal_to / 2)
                advantage = (win_percentage- 0.5) * 2
                row_data = {
                    'distinguisher': 'KLDScore',
                    'advantage': advantage,
                    'forget_set_size': forget_set_size.split('_')[-1],
                    'win_percentage': win_percentage
                }
                adv_data.append(row_data)
            
        # ul_approach == distinguisher_data['model']
        # if ul_approach == 'control':
        #     ul_data = distinguisher_data[distinguisher_data['forget_set'] == forget_set_size]
        #     mean_scores = {'mean': np.mean(ul_data['score']), 'std': np.std(ul_data['score']), 'method': ul_approach, 'distinguiser':distinguisher_fp.parts[-1][:-4]}
        #     all_means.append(mean_scores)
        #     continue
        # else:
        #     ul_data = distinguisher_data[distinguisher_data['forget_set'] == ul_approach]
        #     greater_than = (ul_data['score'].values > control_data['score'].values).sum() / len(ul_data['score'].values)
        #     less_than = (ul_data['score'].values < control_data['score'].values).sum() / len(ul_data['score'].values)
        #     advantage = (max(greater_than, less_than)- 0.5) * 2
        #     row_data = {
        #         'distinguisher': distinguisher_name,
        #         'advantage': advantage,
        #         'unlearning': ul_approach
        #     }
        #     # print(greater_than, less_than)
        #     graph_data.append(row_data)
        #     mean_scores = {'mean': np.mean(ul_data['score']), 'std': np.std(ul_data['score']), 'method': ul_approach, 'distinguiser':distinguisher_fp.parts[-1][:-4]}
        #     all_means.append(mean_scores)
graph_df = pd.DataFrame.from_dict(graph_data)
adv_data = pd.DataFrame.from_dict(adv_data)


In [52]:
fig = px.line(
    graph_data,
    x='forget_set_size',
    y='score',
    color='model',
    error_y='std',
    # title=f"Comparison of Certified Removal Sigma against {distinguisher.replace('_', ' ').upper()} Distinguisher Scores",
    width=900,
    height=450,
    markers=True,
    labels={
        'model': 'Unlearning Method',
        'score': "KLDScore",
        'forget_set_size': "Size of the Forget Set (# of examples)"
    },
    category_orders={'model': ['Control (Retrain)', 'Certified Deep Unlearning', ]}
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_yaxes(minallowed=-0.1)
fig.write_image('images/kld_cdu_forget_set_size.png')
fig.write_image('images/kld_cdu_forget_set_size.pdf')

fig.show()

In [53]:
fig = px.bar(adv_data, x="forget_set_size", y="win_percentage", barmode="stack", color="forget_set_size",
    width=900,
    height=450,
    labels = {
        'win_percentage': 'Success Rate',
        'forget_set_size': "Size of the Forget Set (# of examples)"
    }
)
fig.update_yaxes(minallowed=0.5)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    showlegend=False,
)
fig.write_image('images/adv_cdu_forget_set_size.png')
fig.write_image('images/adv_cdu_forget_set_size.pdf')


fig.show()

In [57]:
from pathlib import Path
from argparse import ArgumentParser
import pandas as pd
import numpy as np

experiment_dir = '/fendlnm1/mirror-mirror/heuristic_random/'
exp_dir = Path(experiment_dir)
distinguishers = [fp for fp in exp_dir.iterdir() if str(fp).endswith('csv')]
# print(distinguishers)
graph_data = []
all_means = []
adv_data = []
for distinguisher_fp in distinguishers:
    distinguisher_data = pd.read_csv(distinguisher_fp)
    # unlearning_approaches = distinguisher_data['model'].unique()
    # for ul_approach in unlearning_approaches:
    for forget_set_size in distinguisher_data['forget_set'].unique():
        control_data = distinguisher_data[np.array(distinguisher_data['model'] == 'control') & np.array(distinguisher_data['forget_set'] == forget_set_size)]
        for ul_approach in distinguisher_data['model'].unique():
            ul_string = name_mapping[ul_approach]
            # ul_string = ul_approach
            condition = np.array(distinguisher_data['forget_set'] == forget_set_size) & np.array(distinguisher_data['model'] == ul_approach)
            ul_data = distinguisher_data[condition]
            graph_data.append({
                'score': ul_data['score'].mean(),
                'model': ul_string,
                'forget_set_size': forget_set_size.split('_')[-1],
                'std': ul_data['score'].std(),
            })
            if ul_approach != 'control':
                greater_than = (ul_data['score'].values > control_data['score'].values).sum() / len(ul_data['score'].values)
                less_than = (ul_data['score'].values < control_data['score'].values).sum() / len(ul_data['score'].values)
                equal_to = (ul_data['score'].values == control_data['score'].values).sum() / len(ul_data['score'].values)
                win_percentage =  max(greater_than, less_than) + (equal_to / 2)
                print(win_percentage)
                advantage = (win_percentage - 0.5) * 2
                row_data = {
                    'distinguisher': 'MIAScore',
                    'advantage': advantage,
                    'forget_set_size': forget_set_size.split('_')[-1],
                    'model':ul_string,
                    'win_percentage': win_percentage,
                }
                adv_data.append(row_data)

graph_df = pd.DataFrame.from_dict(graph_data)
adv_data = pd.DataFrame.from_dict(adv_data)


0.70703125
0.8515625
0.66015625
0.83984375
0.96875
0.6953125
0.96875
1.0
0.6796875
0.98828125
1.0
0.734375
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [55]:
fig = px.line(
    graph_data,
    x='forget_set_size',
    y='score',
    color='model',
    error_y='std',
    # title=f"Comparison of Certified Removal Sigma against {distinguisher.replace('_', ' ').upper()} Distinguisher Scores",
    width=900,
    height=450,
    markers=True,
    labels={
        'model': 'Unlearning Method',
        'score': "MIAScore",
        'forget_set_size': "Size of the Forget Set (# of examples)"
    },
    category_orders={'model': ['Control (Retrain)', 'Amnesiac', 'Bad Teacher', 'SSD']}
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_yaxes(minallowed=-0.1)
fig.write_image('images/kld_heuristic_forget_set_size.png')
fig.write_image('images/kld_heuristic_forget_set_size.pdf')

fig.show()

In [56]:
fig = px.bar(adv_data, x="forget_set_size", y="win_percentage", barmode="group", color='model',
    width=900,
    height=450,
    labels = {
        'win_percentage': 'Success Rate',
        'forget_set_size': "Size of the Forget Set (# of examples)",
        'model': 'Unlearning Methods'
    },
        category_orders={'model': ['Control (Retrain)', 'Amnesiac', 'Bad Teacher', 'SSD']}
)
fig.update_yaxes(minallowed=0.0)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.write_image('images/adv_heuristic_forget_set_size.png')
fig.write_image('images/adv_heuristic_forget_set_size.pdf')

fig.show()

In [ ]:
# distinguisher_data['Forget Set Size'] = [x.split('_')[-1] for x in distinguisher_data['forget_set']]
# distinguisher_data.keys()